In [1]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd

# Section 1: Mean-Variance-Skewness-Kurtosis Approach

We have some data and need to compute the utility of various assets according to a new utility function:

$$U(x) = \mathbb E[x] - \frac{A_1}{2} \text{Var}(x) + A_2 \text{Skew}(x) - A_3 \text{Kurt}(x)$$

We can think of this as **the** Mean-Variance utility function, but we usually set $A_2$ and $A_3$ to zero.

Here's the data:

In [2]:
data = np.array([[0.0058, 0.0452, -2.14, 14.91],
                 [0.0088, 0.0056, 0.29, 4.76],
                 [0.0093, 0.0462, -0.21, 3.39]])
df = pd.DataFrame(data, index=['MSCI Emerging', 'Long/Short', 'MSCI World'],
                 columns = ['Mean', 'Volatility', 'Skewness', 'Kurtosis'])
df

,Mean,Volatility,Skewness,Kurtosis
MSCI Emerging,0.0058,0.0452,-2.14,14.91
Long/Short,0.0088,0.0056,0.29,4.76
MSCI World,0.0093,0.0462,-0.21,3.39


## Question 1.1

Which type of fund do we prefer if our MV utility has parameters $A_1 = 3, A_2 = 0, A_3 = 0$

We apply the function to data and find that **Long/Short** has the highest utility.

- Remark: Remember that utility is a meaningless concept. It can even be negative, but this doesn't matter at all. We only care about the relative rankings between choices.

In [3]:
def MV_Utility(data, A1, A2=0, A3=0):
    """
    Function to compute the mean-variance utility of an asset
    """
    return data['Mean'] - 0.5 * A1 * data['Volatility'] ** 2 + A2 * data['Skewness'] - A3 * data['Kurtosis']

df.apply(MV_Utility, axis=1, A1=3)

MSCI Emerging    0.002735
Long/Short       0.008753
MSCI World       0.006098
dtype: float64

## Question 1.2

How about if you have $A_1 = 3, A_2 = 10, A_3 = 1$?

Applying the formula again, we still prefer the Long/Short portfolio. Maybe this isn't surprising, because the weight on Skewness is so large! 

In [4]:
df.apply(MV_Utility, axis=1, A1=3, A2=10, A3=1)

MSCI Emerging   -36.307265
Long/Short       -1.851247
MSCI World       -5.483902
dtype: float64

# Section 2: Risk Aversion and Risk Measures

We are told that our portfolio:

1. Has an overall historical return of 10%
2. Has a loss probability of 15%
3. The risk-free rate is 3%

Assume that the portfolio has returns $\tilde r_p \sim N(\mu_p, \sigma^2_p)$. 

## Question 2.1:

What is the expected volatility of the portfolio?

We know that: 
1. 15% of the mass of the normal should be negative
2. The mean of the distributiion is 10%

So we just need to set $\mu = 0.1$, then find $\sigma$ such that condition 1 is met. The question asks us to use a z-table to find the answer (why not a slide ruler...)

The z-table tells us $P[Z < z]$, where $Z \sim N(0, 1)$ and $z$ is some number of interest. For us, $z = 0$, because we're interested in the negative mass. We want to find a value in the table such that $P[Z < z] = 0.15$. 

Since we're only given positive values, we have to exploit the fact that the normal distribution is totally symmetrical. So if we find $P[Z < z] = 0.85$ then make it negative, we'll have $P[Z < z] = 0.15$. 

Looking up 0.85 in the table, we get a z-score of 1.04, so our target z-score is -1.04.  

Now we just need to find the sigma associated with this z-score, using the formula:

$$Z = \frac{x - \mu}{\sigma}$$

Where $x$ is the critical value we're interested in (0 in our case). Plugging in numbers:

$$-1.04 = \frac{-0.1}{\sigma} \implies \sigma = \frac{0.1}{1.04} = 0.096153$$

In [5]:
# Check out answer by asking for the CDF for a normal with the calculated 
# parameters. It's pretty close!
d = stats.norm(loc=0.1, scale=0.096153)
d.cdf(0)

0.14916782434020298

## Question 2.2

A client wants to invest 60% of his wealth in the risky asset and 40% of his wealth in the safe asset. What is his risk aversion coefficient $A_1$ (assume $A_2 = 0, A_3 = 0$)

To combine the portfolios, just take a linear combination of the underlying random variables:

$$Z = \tau \tilde r_p + (1 - \tau) \tilde r_f$$

Where $\tau$ is the fraction of wealth the investor wants to put into the risky stock. 

Since $\tilde r_f$ is deterministic (it's not a probability distribution), $Z$ will follow a normal distribution. This is because of the following rules:

Given a random variable $X \sim N(\mu, \sigma^2)$, a new random variable $Y = aX + b$ will have distribution $Y \sim N(a\mu + b, a^2 \sigma^2)$

In this case, we have $a = \tau$ and $b = (1 - \tau) \tilde r_f$.

The problem of the investor was to choose a portfolio weight $\tau$ to maximize his utility. Given a choice of $\tau$, we get the following formulas for the mean and variance of the portfolio (this comes from the normal rule stated above):

$$\begin{align} \mu_Z &= \tau \mu_p + (1 - \tau)\tilde r_f \\
                \sigma^2_Z &= \tau ^2 \sigma^2_p \end{align}$$

Now we can plug these moments into the MV Utility function and find the best possible $\tau$. The investor solved this problem:
$$\max_{\tau} \tau \mu_p + (1 - \tau)\tilde r_f - \frac{A_1}{2}(\tau ^2 \sigma^2_p)$$

Find the maximum of this function by taking the derivative and setting it equal to zero:

$$\begin{align} \mu_p - \tilde r_f - A_1 \sigma^2_p \tau &= 0 \\
                -A_1 \sigma^2_p \tau &= \tilde r_f - \mu_p \\
                A_1 &= \frac{\mu_p - \tilde r_f}{\tau \sigma^2_p} \end{align}$$

We know that the investor chose $\tau = 0.6$, so we can plug in and solve for $A_1$:

$$\begin{align} A_1 &= \frac{0.1 - 0.03}{0.096^2 \cdot 0.6} \\
              A_1 &= 12.65 \end{align}$$

## Question 2.3

What distribution does $Z$ follow? Find the mean and variance of $Z$.

We already said that $Z$ is Normally distributed with the following parameters: 

$$\begin{align} \mu_Z &= \tau \mu_p + (1 - \tau)\tilde r_f \\
                \sigma^2_Z &= \tau ^2 \sigma^2_p \end{align}$$

Remember that this is because neither $\tau$ nor $\tilde r_f$ are random variables! 

(The sum of two random variables, $X + Y$ with $X \sim N(\cdot), Y \sim N(\cdot)$, is also normal, but the formula for the first two moments is different. The product $XY$ is **not** normal.)

Anyway, we just need to plug in $\tau = 0.6, \mu_p = 0.1, \sigma^2_p = 0.096, \tilde r_f = 0.03$. We end up with:

$$ Z \sim N(0.6\mu_p + 0.4\tilde r_f, 0.6^2 \sigma^2_p)$$

Or:

$$ Z \sim N(0.072, 0.0033)$$

In [6]:
# Numerical experiment -- 
# Draw a ton of X ~ N(0.1, 0.096153), then add 0.4 * rf and multiply by 0.6
# This way we can check that the moments we computed are right.

# Here, loc is mu and scale is the std. Size is the number of draws
rp_draws = np.random.normal(loc=0.1, scale=0.096153, size=(100_000))

# Shift and scale
Z_draws = rp_draws * 0.6 + 0.4 * 0.03

# Compute statistics
print(Z_draws.mean())
print(Z_draws.var())

# Looks good!

0.0718407862459894
0.0033136320782721084


## Question 2.4

Find x such that $P[Z \leq x] = 0.05$ and $P[Z \geq x] = 0.01$

We can turn to the trusty z-score table again. This is the same as the first question, but now we know mu and sigma, but seek a critical value.

For the first one, we have to do the symmetry trick again: Find $P[Z \leq z_1] = 0.95$, then make it negative. This gives us $z_1 = -1.65$. 

For the second one, we to take the compliment. Instead of looking up $P[Z \geq z_2] = 0.01$, we look up $P[Z \leq z_2] = 0.99$ This will be exactly the same value (because it's the same cutoff, just measuring from the left or the right). Anyway we find $z_2 = 2.33$.

So these two numbers are the z-scores. To adjust them, we just shift and scale by our distribution's parameters using:

$$ z = \frac{x - \mu}{\sigma}$$

We have to solve for x. Quick algebraic rearrangement:

$$ x = \sigma z + \mu$$

So we get:

$$\begin{align}
x_1 &= (0.057)(-1.65) + 0.072 =  -0.0233 \\
x_2 &= (0.057)(2.33) + 0.072 = 0.2058
\end{align}$$



In [7]:
# Sanity check: compute the CDF of x1 and check that it's 5%:
d = stats.norm(loc=0.072, scale=np.sqrt(0.0033))
d.cdf(-0.0233)

0.04856192024984875

In [8]:
# Sanity check: compute the SF (that's 1 - CDF) of x2 and check that it's 1%:
d.sf(0.2058)

0.009925321048615609

# Section 3: Risk Measures

We're going to manage a fund worth €200 billion. Every wear we need to generate €14 billion. Over the "long run", we can't let the fund dip below the initial capital. 

We can choose to invest in:

1. A risky asset $\tilde r$ with $\mathbb E [\tilde r] = 0.1$ and $\text{Std}(\tilde r) = 0.12$.
2. A risk-free asset $\tilde r_f = 0.03$

## Question 3.1

Choose portfolio weights $\tau$ to ensure that, on average, we can make our €12 billion payments.

Well, first we should compute how much return we need to generate each year:

$$\bar r = \frac{14}{200} = 0.07$$

So we need the expected value of the portfolio to be 7%. We know the returns of a portfolio is just a linear combination of the returns of the components, so we just find weight $\tau$ such that:

$$ \tilde r_\tau = \tau \tilde r_p + (1 - \tau) \tilde r_f$$

Rearrange and solve for $\tau$:

$$\tau = \frac{\tilde r_\tau - \tilde r_f}{\tilde r_p - \tilde r_f}$$

Or:

$$\tau = \frac{0.07 - 0.03}{0.1 - 0.03} = \frac{0.04}{0.07} = 0.5714$$

So we should put about 43% of our money into the risky asset, and the remaining 57% into the risk-free asset.

In [9]:
r_p = 0.1
sigma_p = 0.12
r_f = 0.03
r_tau = 14 / 200

tau = (r_tau - r_f) / (r_p - r_f)
tau

0.5714285714285715

In [10]:
# Sanity check: simulation study
# Draw a ton of risky returns from N(0.1, 0.12). Find the average return with 
# tau = 0.571 -- it should be close to 0.07

r_draws = np.random.normal(loc=r_p, scale=sigma_p, size=(1_000_000))
np.mean(tau * r_draws + (1 - tau) * r_f)

# Hooray!

0.07013755775735607

## Question 3.2

If the risky returns are normally distributed, what is the probability to have a negative return? 

First we need to compute the moments of the combined distribution. This comes from the equations above:

$$\begin{align} \mu_Z &= \tau \mu_p + (1 - \tau)\tilde r_f \\
                \sigma^2_Z &= \tau ^2 \sigma^2_p \end{align}$$

We already know that $\mu_Z = 0.07$, so we just need to compute $\sigma_Z = \tau \sigma_p = 0.571 \cdot 0.12 = 0.0686$

Now we need to know the $P[X \leq 0] = P[Z \leq \frac{-\mu}{\sigma}] = P[Z \leq -1.021]$. 

So we do the absolute value thing. $P[Z \leq 1.021] = 0.8461$, so $P[Z \leq -1.021] = 1 - 0.8461 = 0.1539$

In [11]:
# Compute sigma Z
sigma_Z = tau * sigma_p
print(f'σ_Z = {sigma_Z}')

# Compute -mu/sigma
print(f'-μ_Z / σ_Z = {-r_tau / sigma_Z}')

p_negative = 1 - 0.8461
print(f'P(Negative return) = {p_negative}')

σ_Z = 0.06857142857142857
-μ_Z / σ_Z = -1.0208333333333335
P(Negative return) = 0.15390000000000004


In [12]:
# Sanity check: compute the CDF of zero given parameters
d = stats.norm(loc=r_tau, scale=sigma_Z)
d.cdf(0)

0.1536667043960181

So we have about a 12% chance of getting a negative return. We can think of this as a Bernoulli distributed random variable, with probability of "success" (i.e. seeing the negative output) as 12%.

Fun fact: the number of Bernoulli draws required before seeing a success follows a [geometric distribution](https://en.wikipedia.org/wiki/Geometric_distribution). The probability that we require $k$ draws to get a single success is:

$$P(X = k) = (1 - p)^{k-1} p$$

This formula should look very reasonable: We fail (with probably $1 - p$) $k-1$ times, then succeed (with probability $p$). So it's just counting the Bernoulli trials.

Anyway, given $X \sim \text{Geometric}(p)$, $\mathbb E[X] = \frac{1}{p}$.

So on average, we expect our portfolio to lose money every $\frac{1}{0.154} = 6.5$ years

In [13]:
print(f'Expected years before negative return: {1 / p_negative}')

Expected years before negative return: 6.497725795971409


## Question 3.3

Re-do question 3.2, but assume returns are Student T distributed, with $\nu = 3$.

The Student T has the same shifting and scaling rules as the Normal, so we can just recycle our answers of $\mu_Z = 0.07$ and $\sigma_Z = 0.06857$. 

What changes is the probabilities. We have to go looking in the T-table instead of the Z-table for $P[Z \leq \frac{-\mu_Z}{\sigma_Z} = -1.021]$. Doing the absolute value thing and checking the row for $\nu = 3$, we find it's something like $1 - 0.81 = 0.19$.

So the number of years before a negative return is $\frac{1}{0.19} = 5.2$ years

In [14]:
1 / 0.19

5.2631578947368425

In [15]:
# Sanity check: precise CDF for a student T
d = stats.t(df=3, loc=r_tau, scale=sigma_Z)
d.cdf(0)

# The guess of 0.19 was really good!

0.1912385615598606

## Question 3.4

Assuming returns are Gaussian/Student T, what should the portfolio weight $\tau$ be, such that the expected probablity of a loss is every 30 years?

We need to work backwards. We want to choose $p$ such that $30 = \frac{1}{p}$, or $p = \frac{1}{30}$

So we want $P[Z \leq z] = \frac{1}{30} = 0.033$. We don't have small probabilities in the Z-table, so we have to instead look up $1 - 0.033 = 0.966$. We're doing a reverse lookup, so look for the closest probability to 0.966 then read out the associated z-score. It ends up being 1.83, but we need the right side, not the left side, so make it negative: $z = -1.83$. 

(This operation is called the **inverse CDF**, or the **probit function**)

Now that's a Z-score, we need to translate it to our distribution, which we can do with the usual formula:

$$z = \frac{x - \mu}{\sigma}$$

Solve for $x$:

$$ x = \mu + \sigma z$$

(You should recoginize this formula as a shift and a scaling of a standard normal! Remember that we always have $Z \sim N(0, 1)$, and if $Y = \mu + \sigma Z$ then $Y \sim N(\mu, \sigma^2)$)

What is $x$? Acutally it should be zero -- remember what we're doing: we're going to choose $\mu$ and $\sigma$ (by choosing weight $\tau$) so that there's a negative return every 30 years. We found the z-score associated with a 1-in-30 years event, now we need to find the $\mu$ and $\sigma$ that make it happen when $X < x$.

Ok, so sub in the formula for $r_Z$ and $\sigma_Z$:

$$0 = \tau \tilde r_p + (1 - \tau) \tilde r_f + \tau \sigma_p z $$

And solve for tau:

$$ \begin{align}
\tau & = -\frac{r_f}{r_p - r_f + \sigma_p z} \\
\tau &= -\frac{0.03}{0.1 - 0.03 - 0.12 \cdot 1.83} \\
\tau &= 0.2 \end{align}$$


In [16]:
d = stats.norm(loc=0, scale=1)
# In scipy, the inverse cdf is called the ppf
z = d.ppf(1/30)
print(f'Z-score associated with p = 1 / 30: {z}')
tau_30_year_gaussian = -r_f / (r_p - r_f + sigma_p * z)
print(f'Portfolio weight for negative returns every 30 years (Guassian): {tau_30_year_gaussian}')
# tau = 0.03 / (0.03 - d.ppf(1/30) * 0.0026 ** 0.5 - 0.06)
# tau

Z-score associated with p = 1 / 30: -1.8339146358159146
Portfolio weight for negative returns every 30 years (Guassian): 0.19990703483549174


For the Student-T distribution, all the math remains the same, we just have to change the inverse CDF function. Use scipy since the table in the back of the pamphlet isn't very percise:

In [17]:
d = stats.t(df=3, loc=0, scale=1)
z = d.ppf(1/30)
print(f'Student T "Z"-score associated with p = 1 / 30: {z}')
tau_30_year_t = -r_f / (r_p - r_f + sigma_p * z)
print(f'Portfolio weight for negative returns every 30 years (Student T): {tau_30_year_t}')


Student T "Z"-score associated with p = 1 / 30: -2.8213010887017584
Portfolio weight for negative returns every 30 years (Student T): 0.11170849061623045


## Question 3.5

At the 1-in-30 year portfolios weights, what is the average amount we could pay teachers? 

First, compute expected returns on the portfolios given the 1-in-30 year weights

In [18]:
r_30_norm = tau_30_year_gaussian * r_p + (1 - tau_30_year_gaussian) * r_f
r_30_t = tau_30_year_t * r_p + (1 - tau_30_year_t) * r_f

print(f'Gaussian porfolio expected return: {r_30_norm}')
print(f'T porfolio expected return: {r_30_t}')

Gaussian porfolio expected return: 0.043993492438484424
T porfolio expected return: 0.03781959434313613


Multiply that by the assets under management to get our expected earnings

In [19]:
print(f'Expected income from investment (normal): €{r_30_norm * 200:0.2f} billion')
print(f'Expected income from investment (t): €{r_30_t* 200:0.2f} billion')

Expected income from investment (normal): €8.80 billion
Expected income from investment (t): €7.56 billion


So we could afford to pay the teachers either 8.8 or 7.56 billion, depending on our distributional assumptions. 

Because we controled risk, we had to leave profits on the table. This is an example of a "no free lunch" situation.

# Bonus Questions

**Long-Short Strategy and Utility Maximization**

We manage a fund that buys over-valued assets short and under-valued assets long. We have $W_0$ wealth under management. The assets have correlated risk $\rho$. Here's the assets we have access to, plus a riskless asset that pays 3%. 

In [20]:
data = np.array([[10, 0.05, 0.15],
                 [100, 0.15, 0.2]])
df = pd.DataFrame(data, index=['Asset 1', 'Asset 2'], columns=['price', 'mean', 'std'])
df

,price,mean,std
Asset 1,10.0,0.05,0.15
Asset 2,100.0,0.15,0.20


## Question 4.1

You believe that asset 1 is over-valued and asset 2 is under-valued. 

Which asset are you going to sell, and which are you going to buy?

Well, since I think asset 1 is over-valued, I'm going to sell that (go short), and I'm going to buy asset 2 (go long)

## Question 4.2

Determine the number of assets 1 and 2, $n_1$ and $n_2$, respectively, purchased and sold in the transaction. Assume that all proceeds from the shorts are used to take long positions. 

This is asking for the so-called "self-financing constraint". We just note that the money we generate from selling -- $n_1 P_1$ -- is equal to the money we spend from buying -- $n_2 P_2$. Putting it together:

$$n_1 P_1 + n_2 P_2 = 0$$

## Question 4.3 

What is the expected return on the portfolio?

We have three sources of income:

1. Risk-free returns on initial wealth -- $W_0(1 + \tilde r_f)$
2. Returns from short-selling asset 1 -- $(1 + \tilde r_1) n_1 P_1$
3. Returns from buying and holding asset 2 -- $(1 + \tilde r_2) n_2 P_2$

We begin with $W_0$, so we want the change in wealth from that point of comparison:

$$r_p = \frac{W_0(1 + \tilde r_f) + (1 + \tilde r_1) n_1 P_1 + (1 + \tilde r_2) n_2 P_2}{W_0} - 1$$

Expand the fraction:

$$ r_p = (1 + \tilde r_f) + \frac{(1 + \tilde r_1) n_1 P_1}{W_0} + \frac{(1 + \tilde r_2) n_2 P_2}{W_0} - 1 $$

We're asked to define the size of the long-short position as $\delta = \frac{n_2 P_2 }{W_0}$:

$$ r_p = \tilde r_f + \frac{(1 + \tilde r_1) n_1 P_1}{W_0} + \delta (1 + \tilde r_2)$$

Re-write $n_1 P_1$ as $-n_2 P_2$ (this comes from the self-financing constraint). That will let us make anther delta substitution:

$$\begin{align}
r_p &= \tilde r_f -\frac{(1 + \tilde r_1) n_2 P_2}{W_0} + \delta (1 + \tilde r_2) \\
r_p &= \tilde r_f -\delta(1 + \tilde r_1) + \delta(1 + \tilde r_2) \\
r_p &= \tilde r_f + \delta(\tilde r_2 - \tilde r_1)
\end{align}
$$


## Question 4.4 

Determine the expected return and volatility of this portfolio.

We need to know the [convolution of normals formula](https://en.wikipedia.org/wiki/Sum_of_normally_distributed_random_variables). Given two **independent** random variables $X \sim N(\mu_X, \sigma_X^2)$ and $Y \sim N(\mu_Y, \sigma_Y^2)$, then a new random variable $Z = X + Y$ will be $Z \sim N(\mu_X + \mu_Y, \sigma_X^2 + \sigma_Y^2)$

(Exercise for the motivated reader: Use the rules of linear transfomation of Normals to get the result for $Z = X - Y$ by considering $Z = X + U$ and $U = aY$ with $a = -1$)

Actually though, we were told that $\tilde r_1$ and $\tilde r_2$ are correlated, so actually we will have $Z = \tilde r_2 - \tilde r_1$ will be $Z \sim N(\mu_2 - \mu_1, \sigma_1^2 + \sigma_2^2 - 2\rho\sigma_1\sigma_2)$ 

But that's not quite our porfolio return, we also need to shift by $\tilde r_f$ and scale by $\delta$. We have to use those formulas for shifting and scaling a normal distribution again (scroll up to check them again if you forgot). We will end up with the final answer:

$$\begin{align}r_p &\sim N(\mu_p, \sigma^2_p) \\
                \mu_p &= \tilde r_f + \delta(\mu_2 - \mu_1) \\
                \sigma_p^2 &= \delta^2(\sigma_1^2 + \sigma_2^2 - 2\rho\sigma_1\sigma_2)
\end{align}$$
                

## Question 4.5

Determine the optimal strategy formula for long and short positiosn to maximize the mean-variance utility function

So our utility function is:

$$ U = \mu_p - \frac{A}{2} \sigma^2_p$$

And we want to maximize this by choosing $\delta$. Why delta? Well actually we want to choose quantities $n_1,n_2$, and we could set up a constrained optimization with the self-financing constraint... or we can just realize that $\delta$ compeltely summarizes all this information. Given $\delta$, we can work backwards and get $n_1$ and $n_2$. So maximizing with respect to $\delta$ saves a lot of steps.

(Going through those steps is left as an exercise to the motivated reader)

$$\max_{\delta} \tilde r_f + \delta(\mu_2 - \mu_1) - \frac{A}{2}\delta^2(\sigma_1^2 + \sigma_2^2 - 2\rho\sigma_1\sigma_2)$$

Do the usual maximization dance: derivative, set zero, solve.

$$\begin{align}
(\mu_2 - \mu_1) - A\delta(\sigma_1^2 + \sigma_2^2 - 2\rho\sigma_1\sigma_2) &= 0 \\
\delta &= \frac{\mu_2 - \mu_1}{A(\sigma_1^2 + \sigma_2^2 - 2\rho\sigma_1\sigma_2)}
\end{align}$$

## Question 4.6

Calculate a numerical value for $\delta$ when $\rho = 0.1$ and $A = 3$

In [21]:
def delta_star(A, rho, mu1, mu2, sigma1, sigma2):
    numer = mu2 - mu1
    denom = A * (sigma1 ** 2 + sigma2 ** 2 - 2 * rho * sigma1 * sigma2)
    return numer / denom

In [22]:
mu1, mu2 = df['mean'].values
sigma1, sigma2 = df['std'].values

δ_star = delta_star(A=3, rho=0.1, mu1=mu1, mu2=mu2, sigma1=sigma1, sigma2=sigma2)
print(f'Optimum alllocation to Asset 2 when A=3, ρ=0.1: {δ_star:0.3%}')

Optimum alllocation to Asset 2 when A=3, ρ=0.1: 58.997%


## Question 4.7

I disagree with this question somewhat. What we **really** want is a negative correlation between the two assets. This implies that when one goes up, the other goes down, which would augment the expected returns of our strategy (since we're essentailly betting on the two assets diverging).

If we suppose that we can't have that, we might settle for a low correlation coefficient. At any rate, the point is that what we really don't want is strong correlation, because then we won't ever get the "separation" we seek. That is, if the assets are strongly correlated, one position or the other will always be losing us money.

## Question 4.8

Assume $\rho = 0$. Compute comparative statics for $\delta^\star$ with respect to $A, \mu_2, \sigma_1, \sigma_2$

In [23]:
# I'll use sympy to handle the derivatives
import sympy as sp
A, mu1, mu2, sigma1, sigma2 = sp.symbols('A, mu_1, mu_2, sigma_1, sigma_2')
d_star = sp.Symbol('\delta^\star')
d_expr = delta_star(A, 0, mu1, mu2, sigma1, sigma2)

In [24]:
sp.Eq(d_star, d_expr)

Eq(\delta^\star, (-mu_1 + mu_2)/(A*(sigma_1**2 + sigma_2**2)))

### A

A itself is always positive (as well as the variances), so the denominator is always postitve. The sign of the numerator depends on the difference in the expected returns of the two assets. In our case, we have $\mu_1 < \mu_2$, so this expression is strictly negative. As we become more risk averse, we reduce our long-short position.  

In [25]:
sp.Eq(sp.diff(d_star, A, evaluate=False), d_expr.diff(A).simplify())

Eq(Derivative(\delta^\star, A), (mu_1 - mu_2)/(A**2*(sigma_1**2 + sigma_2**2)))

### $\mu_2$

Recall that $\mu_2$ is the return on our long position, so we want it to go higher. As it does, we take a larger long-short position. (The reverse would be true for $\mu_1$)

In [26]:
sp.Eq(sp.diff(d_star, mu2, evaluate=False), d_expr.diff(mu2).simplify())

Eq(Derivative(\delta^\star, mu_2), 1/(A*(sigma_1**2 + sigma_2**2)))

### $\sigma_1$, $\sigma_2$

Analyis of these are symmetrical.

Once agin, because $\mu_2 > \mu_1$, this expression is strictly negative. This makes sense, because, holding risk tolerance and returns fixed, we expect an investor to take a smaller position if the risk increases.

In [27]:
sp.Eq(sp.diff(d_star, sigma1, evaluate=False), d_expr.diff(sigma1).simplify())

Eq(Derivative(\delta^\star, sigma_1), 2*sigma_1*(mu_1 - mu_2)/(A*(sigma_1**2 + sigma_2**2)**2))

In [28]:
sp.Eq(sp.diff(d_star, sigma2, evaluate=False), d_expr.diff(sigma2).simplify())

Eq(Derivative(\delta^\star, sigma_2), 2*sigma_2*(mu_1 - mu_2)/(A*(sigma_1**2 + sigma_2**2)**2))

## Question 4.9

We're going to change the financing structure for the next three quetsions. We no longer have the self-financing constraint. We're going to:

1. Have $W_0$ invested in the risk-free asset that yields $\tilde r_f$
2. We will sell short the over-valued Asset 1, but then invest the proceeds $n_1 P_1$ into the risk-free asset
3. We will borrow money at the market rate $r_f$ to finance our long position of the undervalued Asset 2.

Other elements of the setup remain the same.

**Question:** Express returns of the portfolio using $\delta_1 = -\frac{n_1 P_1}{W_0}$ and $\delta_2 = -\frac{n_2 P_2}{W_0}$

(__Jesse Note__: I think maybe it should read $\delta_2 = \frac{n_2 P_2}{W_0}$, otherwise the final expression doesn't make sense.)

We need to think about our income streams one at a time, as before. We make:

1. $(1 + r_f) W_0$ from our initial investments
2. $-n_1 P_1 (1 + r_f)$ from our re-investment of short-sale profits (negative because we think of $n_1$ as negative, so this needs to be positive)
3. $n_1 P_1 (1 + r_1)$ from holding the short position for 1 period
4. $n_2 P_2 (1 + r_2)$ from holding the long position for 1 period
5. $-n_2 P_2 (1 + r_f)$ is interest on the loan we took to fund the long position

As before, add this all together and find the percent change from $W_0$ to get the total return:

$$r_p = \frac{(1 + r_f) W_0 - n_1 P_1 (1 + r_f) + n_1 P_1 (1 + r_1) + n_2 P_2 (1 + r_2) - n_2 P_2 (1 + r_f)}{W_0} - 1$$

Expand the fraction and gather like terms:

$$\begin{align}
r_p &= \frac{(1 + r_f)W_0}{W_0} - 1 + \frac{(1 + r_1) n_1 P_1 - (1 + r_f) n_1 P_1)}{W_0} + \frac{(1 + r_2) n_2 P_2 - (1 + r_f)n_2 P_2}{W_0}\\
r_p &= r_f + \frac{(r_1 - r_f)n_1 P_1}{W_0} + \frac{(r_2 - r_f)n_2 P_2}{W_0}
\end{align}$$

Lastly use the definitions of $\delta_1$ and $\delta_2$:

$$ r_p = r_f - \delta_1(r_1 - r_f) + \delta_2(r_2 - r_f)$$

## Question 4.10

Choose $\delta_1$ and $\delta_2$ to maximize mean-variance utility.

First, we need the mean and variance of $r_p$. We will proceed in steps.

**1. Find the component distributions**

Define $X \sim N(\mu_1, \sigma_1^2)$. Find the distribution of $\bar X = \delta_1 X + \delta_1 r_f$.

This is just shifting and scaling, so $\bar X \sim N(\delta_1\mu_1 + \delta_1 r_f, \delta_1^2\sigma_1^2)$

Similarly, define $Y \sim N(\mu_2, \sigma_2^2)$. We seek the distribution of $\bar Y = \delta_2 r_2 - \delta_2 r_f$.

By the same logic, $\bar Y \sim N(\delta_2 \mu_2 - \delta_2 r_f, \delta_2^2 \sigma_2^2)$

**2. Add the components with convolution formula**

Don't worry about the $r_f$ out front for now. We need to get:

$$ Z = \bar Y - \bar X$$

From the convolution formula, we know the distribution of $Z$ will be:

$$Z \sim N(\mu_Y - \mu_X, \sigma^2_X + \sigma^2_Y - 2 \rho \sigma_X \sigma_Y)$$

Plugging in values, we find:
$$\begin{align}
\mu_Z &= \delta_2 \mu_2 - \delta_2 r_f - \delta_1 \mu_1 + \delta_1 r_f \\
\sigma_Z^2 &= \delta_2^2 \sigma_2^2 + \delta_1^2\sigma_1^2 - 2 \rho \delta_2\sigma_2 \delta_1 \sigma_1 \end{align}
$$

We can clean up $\mu_Z$ a bit:

$$\mu_Z = -\delta_1(\mu_1 - r_f) + \delta_2(\mu_2 - r_f)$$

**3. Shift by $r_f$**

The last thing is to include the $r_f$ term we ignored. After all this, we know $r_p = r_f + Z$. This is just a shift by a constant value, so we just add it to the mean (the variance is unchanged). The distribution of $r_p$ is then:

$$\begin{align}r_p &\sim N(\mu_p, \sigma_p^2) \\ 
\mu_p &= r_f - \delta_1(\mu_1 - r_f) + \delta_2(\mu_2 - r_f) \\
\sigma_p^2 &= \delta_2^2 \sigma_2^2 + \delta_1^2\sigma_1^2 - 2 \rho \delta_2\sigma_2 \delta_1 \sigma_1
\end{align}$$

#### Utility Maximization

We're finally ready to maximize utility with respect to $\delta_1$ and $\delta_2$:

$$
\begin{align}
\max_{\delta_1, \delta_2} \mu_p - \frac{A}{2}\sigma^2_p \\
\max_{\delta_1, \delta_2} r_f - \delta_1(\mu_1 - r_f) + \delta_2(\mu_2 - r_f) - \frac{A}{2}(\delta_2^2 \sigma_2^2 + \delta_1^2\sigma_1^2 - 2 \rho \delta_2\sigma_2 \delta_1 \sigma_1)
\end{align}
$$

We will have two first order conditions, one for $\delta_1$ and one for $\delta_2$:

$$\frac{\partial U}{\partial \delta_1} = 0 \implies -\mu_1 - r_f - A (\delta_1\sigma_1^2 - \rho \delta_2\sigma_2\sigma_1) = 0$$

$$\frac{\partial U}{\partial \delta_2} = 0 \implies \mu_2 + r_f - A (\delta_2\sigma_2^2 - \rho \delta_1\sigma_2\sigma_1) = 0$$

We have two equations and two unknowns, so we can solve this system. Re-arrage the first equation to solve for $\delta_1$, and the second to solve for $\delta_2$:

$$\begin{align}
\delta_1 &= -\frac{\mu_1 - r_f}{A \sigma_1^2} + \frac{\sigma_2}{\sigma_1}\rho\delta_2\\
\delta_2 &= \frac{\mu_2 - r_f}{A\sigma_2^2} + \frac{\sigma_1}{\sigma_2}\rho\delta_1
\end{align}$$

Then plug one into the other to get a final expression (I plug $\delta_2$ into $\delta_1$):

$$\begin{align}
\delta_1 &= -\frac{\mu_1 - r_f}{A \sigma_1^2} + \frac{\sigma_2}{\sigma_1}\rho\left(\frac{\mu_2 - r_f}{A\sigma_2^2} + \frac{\sigma_1}{\sigma_2}\rho\delta_1 \right)\\
\delta_1 &= -\frac{\mu_1 - r_f}{A \sigma_1^2} + \rho \frac{\mu_2 - r_f}{A\sigma_1\sigma_2} + \rho^2 \delta_1 \\
(1 - \rho^2)\delta_1 &= -\frac{\mu_1 - r_f}{A \sigma_1^2} + \rho \frac{\mu_2 - r_f}{A\sigma_1\sigma_2} \\
\delta_1 &= -\frac{\mu_1 - r_f}{(1 - \rho^2) A \sigma_1^2} + \frac{\rho}{1 - \rho^2} \frac{\mu_2 - r_f}{A\sigma_1\sigma_2}
\end{align}$$

Notice that the problem would be totally symmeterical if you plugged $\delta_1$ into $\delta_2$ isntead, so you can directly write that answer down by pattern matching:

$$\delta_2 = \frac{\mu_2 - r_f}{(1 - \rho^2) A \sigma_2^2} - \frac{\rho}{1 - \rho^2} \frac{\mu_1 - r_f}{A\sigma_1\sigma_2}$$

## Question 4.11

Set $\rho = 0$ and discuss the formulas for $\delta_1, \delta_2$.

If $\rho = 0$, then:

$$\begin{align}
\delta_1 &= -\frac{\mu_1 - r_f}{A \sigma_1^2} \\
\delta_2 &= \frac{\mu_2 - r_f}{A \sigma_2^2}
\end{align}$$

The shares alloted to each "side" of the long-short position increase if the returns move in the direction we bet (we buy more long if $\mu_2$ goes up, we sell more short if $\mu_1$ does down). We also reduce our positions if the variance (risk) of either asset goes up. Finally, if our appetite for risk goes down (A goes up), we take smaller positions on both sides of the bet. 